<a href="https://www.kaggle.com/code/hanguyen18020434/uet-hackathon-ai-project?scriptVersionId=96585307" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

# Loading data

In [2]:
work_train = pd.read_csv('../input/uet-hackathon-2022-data-science/work_train.csv')
work_test = pd.read_csv('../input/uet-hackathon-2022-data-science/work_test.csv')

info_train = pd.read_csv('../input/uet-hackathon-2022-data-science/info_train.csv')
info_test = pd.read_csv('../input/uet-hackathon-2022-data-science/info_test.csv')

label_train = pd.read_csv('../input/uet-hackathon-2022-data-science/label_train.csv')
label_test = pd.read_csv('../input/uet-hackathon-2022-data-science/label_test.csv')

In [3]:
work_train.head()
work_test.head()
info_train.head()
info_test.head()

,bithYear,gender,address,id_bh
0,1975,FEMALE,số 16 Phan Chu Trinh,116303809
1,1971,MALE,Nam Định,116301808
2,1989,MALE,NaN,131644973
3,1983,FEMALE,NaN,131264273
4,1962,MALE,Hà Nội,113003795


# EDA

In [4]:
info_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27502 entries, 0 to 27501
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   bithYear  27502 non-null  int64 
 1   gender    27502 non-null  object
 2   address   13214 non-null  object
 3   id_bh     27502 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 859.6+ KB


In [5]:
work_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247559 entries, 0 to 247558
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             247559 non-null  int64  
 1   id_bh          247559 non-null  int64  
 2   id_management  247559 non-null  int64  
 3   id_office      235391 non-null  object 
 4   company_type   247559 non-null  int64  
 5   job/role       214102 non-null  object 
 6   from_date      247559 non-null  int64  
 7   to_date        247559 non-null  int64  
 8   employee_lv    247554 non-null  float64
 9   address        183528 non-null  object 
dtypes: float64(1), int64(6), object(3)
memory usage: 18.9+ MB


In [6]:
# info_train['gender'].value_counts()

In [7]:
# info_test['gender'].value_counts()

In [8]:
# work_train['job/role'].value_counts()

In [9]:
# work_test['job/role'].value_counts()

In [10]:
# work_train['address'].value_counts()

In [11]:
# work_test['address'].value_counts()

In [12]:
# info_train['bithYear'].value_counts().sort_index(ascending=True).plot(kind='bar', figsize=(10,5))

In [13]:
# info_test['bithYear'].value_counts().sort_index(ascending=True).plot(kind='bar', figsize=(10,5))

In [14]:
# work_train['company_type'].value_counts().sort_index(ascending=True).plot(kind='bar')

In [15]:
# work_test['company_type'].value_counts().sort_index(ascending=True).plot(kind='bar')

In [16]:
# work_train['employee_lv'].value_counts().sort_index(ascending=True).plot(kind='bar')

In [17]:
# work_test['employee_lv'].value_counts().sort_index(ascending=True).plot(kind='bar')

# Preprocessing and training

In [18]:
#Remove Vietnamese accent and normalize function

def normalize(s):
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[ÍÌỈĨỊ]', 'I', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
    s = re.sub('đ', 'd', s)
    s = re.sub('Đ', 'D', s)
    if type(s) == str:
        s = str(s).strip().lower()
        s = re.sub(' +', " ", s)
    return s

In [19]:
#Categorize address function (Not used in final)

# def address_categorize(s):
#     if type(s) == str:
#         if "ha noi" in s or "hn" in s or "ha no" in s  or "ha n?i" in s or "thµnh phe hµ nei" in s \
#         or "ha tay" in s or "tu liem" in s \
#         or "cau giay" in s or "thanh xuan" in s or "dong da" in s or "dich vong" in s or "ha dong" in s or "nhan chinh" in s \
#         or "dao tan" in s or "kim ma" in s or "thanh cong" in s or "phuc xa" in s or "keangnam" in s \
#         or "nui truc" in s or "lang ha" in s or "nguyen van cu" in s or "thanh oai" in s \
#         or "ba dinh" in s or "doi can" in s or "hoang hoa tham" in s or "yen phu" in s or "lotte center" in s \
#         or "hoang minh giam" in s or "pho bach dang" in s or "viet hung" in s or "pho vong duc" in s \
#         or "lac long quan" in s or "doc tam da" in s or "bo de" in s or "quan nhan" in s or "giang vo" in s \
#         or "nguyen chi thanh" in s or "tho quan" in s or "de la thanh" in s or "khu mieu" in s \
#         or "nam dong" in s or "kham thien" in s or "dang tien dong" in s \
#         or "thai ha" in s or "dang van ngu" in s or "thai thinh" in s or "phuong mai" in s or "tay ho" in s \
#         or "hai ba trung" in s or "van ho " in s or "doi cung" in s or "bach khoa" in s or "vinh tuy" in s \
#         or "mai huong" in s or "hai ba t" in s or "tap the det 8/3" in s or "truong dinh" in s or "tan trieu" in s \
#         or "yen hoa" in s or "quan tho " in s or "nguyen khang" in s or "pham tuan tai" in s \
#         or "me tro thuong" in s or "nguyen ngoc nai" in s or "linh dam" in s or "trung hoa" in s \
#         or "nguyen trai" in s or "nguyen xien" in s or "hoang van thai" in s or "chelsea park" in s or "phu kieu" in s \
#         or "dai thanh" in s or "duong 32" in s or "khu tap the 664" in s or "huynh cung" in s or "yen thi" in s:
#             return "ha noi"
#         elif "dak lak" in s or "daklak" in s or"dakk lak" in s or "tp.bmt" in s:
#             return "dak lak"
#         elif "ba ria" in s or "vung tau" in s or "brvt" in s:
#             return "ba ria vung tau"
#         elif "bac giang" in s or "bg" in s or "bac giang" in s or "yen the" in s or "bac gian" in s or "thon buom" in s:
#             return "bac giang"
#         elif "bac kan" in s or "bac can" in s:
#             return "bac kan"
#         elif "dak nong" in s or "daknong" in s:
#             return "dak nong"
#         elif "hai duong" in s or "an nhan tay" in s:
#             return "hai duong"
#         elif "hoa binh" in s or "hb" in s or "phuong huu nghi" in s:
#             return "hoa binh"
#         elif "hung yen" in s or "hung yon" in s:
#             return "hung yen"
#         elif "lam dong" in s or "ld" in s or "da lat" in s or "bao loc" in s or "duc trong" in s or "dalat" in s:
#             return "lam dong"
#         elif "nghe an" in s or "do luong" in s or "dien chau" in s or "quynh luu" in s or "q.luu" in s or "nghe an" in s:
#             return "nghe an"
#         elif "nam dinh" in s or "ý yen" in s:
#             return "nam dinh"
#         elif "quang ninh" in s or "ha long" in s:
#             return "quang ninh"
#         elif "ho chi minh" in s or "hcm" in s:
#             return "ho chi minh"
#         elif "thai binh" in s or "thon bich du" in s:
#             return "thai binh"
#         elif "thua thien hue" in s or "hue" in s:
#             return "hue"
#         elif "vinh phuc" in s or "vp" in s or "vinh phu" in s:
#             return "vinh phuc"
#         else:
#             return s
#     else:
#         return np.nan

In [20]:
#Categorize job function (Not used in final)

# def job_categorize(s):
#     if type(s) == str:
#         if "cong nhan vien" in s or "nhan vien" in s or "giao dich vien" in s or "van thu" in s \
#         or "thu ky" in s or "tro ly" in s or "nv" in s or "kiem soat vien" in s \
#         or "van phong" in s or "xa vien" in s or "kiem thu vien" in s or "vien chuc" in s \
#         or "kiem lam vien" in s or "chap hanh vien" in s or "hanh chinh" in s or "kiem tra chat luong" in s \
#         or "thu vien" in s or "quan trac vien" in s or "phu trach quan he nguoi tieu dung" in s:
#             return "nhan vien"
#         elif "cong nhan" in s or "may cong nghiep" in s or "san pham" in s or "cn" in s or "khai thac" in s or "tho" in s \
#         or "phet keo de va mu giay" in s or "ve sinh may chai ,day truyen soi ,det ,nhuom" in s or "c.n" in s \
#         or "son, in da va pha che hoa chat de son, in da" in s or "sua chua" in s or "kt 3d" in s \
#         or "cung nhon lap rop mach dien tu " in s or "luu hoa cac san pham cao su" in s or "qc" in s \
#         or "thuy thu" in s or "cat vai trong cong nghe may" in s or "son, in da va pha che hoa chat de son, in da" in s \
#         or "phay" in s or "phet keo mu giay" in s:
#             return "cong nhan"
#         elif "can bo" in s or "can su" in s:
#             return "can bo"
#         elif "chuyen vien" in s or "chuyen gia" in s:
#             return "chuyen gia"
#         elif "giam doc" in s or "tong giam doc" in s:
#             return "giam doc"
#         elif "pho giam doc" in s or "pho tong giam doc" in s or "pho gd" in s or "p. giam doc" in s or "p.giam doc" in s:
#             return "pho giam doc"
#         elif "pho hieu truong" in s or "hieu pho" in s:
#             return "pho hieu truong"
#         elif "to truong" in s or "truong phong" in s or "quan ly" in s or "chuyen truong " in s or "doi truong" in s \
#         or "chu nhiem" in s or "quan ly" in s or "giam sat" in s or "nhom truong" in s or "truong ca" in s \
#         or "truong nhom" in s or "quan doc" in s or "cua hang truong" in s or "thuyen truong" in s \
#         or "kiem soat truong" in s or "doc cong" in s or "chu quan" in s or "tram truong" in s or "ca truong" in s:
#             return "quan ly truong"
#         elif "to pho " in s or "pho truong phong" in s or "pho phong" in s or "pho chanh" in s or "pho chu nhiem" in s \
#         or "pho quan doc" in s or "doi pho" in s or "pho chi cuc truong" in s:
#             return "quan ly pho"
#         elif "bi thu" in s or "uy vien ban thuong vu" in s or "uy vien uy ban kiem tra" in s or "thuong truc dang uy" in s:
#             return "bi thu"
#         elif "chi huy truong quan su " in s or "truong cong an" in s:
#             return "truong quan su"
#         elif "bo doi" in s or "thanh tra vien" in s or "chien sy" in s:
#             return "quan su"
#         elif "ky su" in s or "ky thuat" in s or "kien truc su" in s or "lap trinh vien" in s or "ky thuat" in s:
#             return "ky su"
#         elif "giao vien" in s or "giang vien" in s:
#             return "giao vien"
#         elif "lai xe" in s or "tai xe " in s or "lai" in s or "lx" in s or "phu xe" in s:
#             return "lai xe"
#         elif "ke toan" in s or "kinh te vien" in s or "mau dich vien" in s or "kinh te" in s or "thong ke" in s:
#             return "kinh te vien"
#         elif "lao dong" in s or "ldpt" in s:
#             return "lao dong"
#         elif "y sy" in s or "bac sy" in s or "trinh duoc vien" in s or "duoc sy" in s:
#             return "bac sy"
#         elif "dieu duong" in s or "ho ly" in s or "ho sinh" in s or "y ta" in s or "duoc ta" in s:
#             return "dieu duong"
#         elif "thu quy" in s or "thu kho" in s:
#             return "thu nhan"
#         elif "phong vien" in s or "bien tap vien" in s:
#             return "truyen hinh"
#         elif "ban hang" in s or "kinh doanh" in s:
#             return "kinh doanh"
#         elif "phien dich" in s or "phien dich tieng han" in s:
#             return "phien dich"
#         elif "dia chinh - xay dung " in s:
#             return "xay dung"
#         elif "van hoa - xa hoi" in s:
#             return "van hoa xa hoi"
#         elif "tu phap - ho tich" in s or "luat su" in s or "kiem sat vien" in s or "tu phap ho tich" in s:
#             return "tu phap ho tich"
#         elif "nau an" in s or "phu bep" in s:
#             return "nau an"
#         else:
#             return s
#     else:
#         return np.nan

In [21]:
#Normalize address function (Not used in final)

# def address_normalize(s):
#     s = normalize(s)
#     s = address_categorize(s)
#     return s

In [22]:
#Normalize job function (Not used in final)

# def job_normalize(s):
#     s = normalize(s)
#     s = job_categorize(s)
#     return s

In [23]:
#Remove NaN

info_train["address"] = info_train["address"].replace(np.nan, "viet nam")
info_test["address"] = info_test["address"].replace(np.nan, "viet nam")

work_train.rename(columns = {'address': 'work_address'}, inplace = True)
work_test.rename(columns = {'address': 'work_address'}, inplace = True)
work_train["work_address"] = work_train["work_address"].replace(np.nan, "viet nam")
work_test["work_address"] = work_test["work_address"].replace(np.nan, "viet nam")

work_train["job/role"] = work_train["job/role"].replace(np.nan, "thieu")
work_test["job/role"] = work_test["job/role"].replace(np.nan, "thieu")

In [24]:
#Normalize text data

info_train["address"] = info_train["address"].apply(normalize)
info_test["address"] = info_test["address"].apply(normalize)

work_train["work_address"] = work_train["work_address"].apply(normalize)
work_test["work_address"] = work_test["work_address"].apply(normalize)

work_train["job/role"] = work_train["job/role"].apply(normalize)
work_test["job/role"] = work_test["job/role"].apply(normalize)

In [25]:
#Merge training data and generate features

full_train = pd.merge(left=work_train, right=info_train, how='left', on='id_bh')
train = pd.merge(full_train, label_train, on='id_bh', how="inner")

train_filter = train.sort_values(
    by=['id_bh', 'to_date', 'from_date'], 
    ascending=[True, False, False]
).drop_duplicates(
    subset=['id_bh'], 
    keep='first', 
    ignore_index=True
)

train_filter['age'] = 2022 - train_filter['bithYear'].astype(int) #use age instead of birthYear

train_filter['latest'] = train_filter['to_date'].astype('string').str.slice(0, 4)
train_filter['gap'] = 2022 - train_filter['latest'].astype(int)

train_filter.head()
# train_filter.info()

,id,id_bh,id_management,id_office,company_type,job/role,from_date,to_date,employee_lv,work_address,bithYear,gender,address,label,age,latest,gap
0,4686,100000725,100,HW01180,6,"cvc, tp",20210600,20220400,17.0,hn,1963,MALE,ha noi,5,59,2022,0
1,29905,100007067,2400,TA0002A,1,nhan vien ban xang dau,20190100,20220400,9.0,tp bac giang,1971,MALE,bac giang,4,51,2022,0
2,4503,100007555,109,TI3361I,-1,nhan vien,20211000,20220400,9.0,viet nam,1970,MALE,viet nam,2,52,2022,0
3,2939,100008102,100,HW0013Z,-1,"chuyen vien chinh, uy vien thuong truc uy ban ...",20200800,20220400,18.0,ha noi,1970,MALE,hn,5,52,2022,0
4,8967,100008777,114,HN05360,-1,pct ubnd huyen,20201000,20220400,17.0,ha noi,1965,MALE,viet nam,6,57,2022,0


In [26]:
#Show correlation map of training data

corr = train_filter.corr()
corr.style.background_gradient(cmap='coolwarm')

,id,id_bh,id_management,company_type,from_date,to_date,employee_lv,bithYear,label,age,gap
id,1.000000,0.417835,0.387072,0.020568,0.117980,-0.069366,-0.121784,0.075750,0.021182,-0.075750,0.066256
id_bh,0.417835,1.000000,0.610439,0.034154,0.097342,0.014768,-0.002081,-0.056548,0.151603,0.056548,-0.014777
id_management,0.387072,0.610439,1.000000,-0.046810,0.117374,-0.042032,0.064258,0.071901,0.138424,-0.071901,0.041385
company_type,0.020568,0.034154,-0.046810,1.000000,-0.111265,0.116334,0.040448,-0.263561,0.177132,0.263561,-0.115067
from_date,0.117980,0.097342,0.117374,-0.111265,1.000000,0.063016,0.081172,0.245053,0.123798,-0.245053,-0.062117
to_date,-0.069366,0.014768,-0.042032,0.116334,0.063016,1.000000,-0.011233,-0.168368,0.067351,0.168368,-0.999291
employee_lv,-0.121784,-0.002081,0.064258,0.040448,0.081172,-0.011233,1.000000,-0.093987,0.488439,0.093987,0.011536
bithYear,0.075750,-0.056548,0.071901,-0.263561,0.245053,-0.168368,-0.093987,1.000000,-0.147635,-1.000000,0.165413
label,0.021182,0.151603,0.138424,0.177132,0.123798,0.067351,0.488439,-0.147635,1.000000,0.147635,-0.067327
age,-0.075750,0.056548,-0.071901,0.263561,-0.245053,0.168368,0.093987,-1.000000,0.147635,1.000000,-0.165413


In [27]:
#Configure training data for CatBoost models

categorical_index = [0, 1, 2, 4]
text_index = [7, 8, 9]

# X_train = train_filter[['id_management', 'id_office', 'company_type', 'employee_lv', 'gender', 'age', 'gap']] #no text features
X_train = train_filter[['id_management', 'id_office', 'company_type', 'employee_lv', 'gender', 'age', 'gap', 'work_address', 'address', 'job/role']] #include text features

Y_train = train_filter['label']

In [28]:
rm -r ./catboost_info

rm: cannot remove './catboost_info': No such file or directory


In [29]:
#Configure CatBoost parameters and models

import catboost
from catboost import cv, Pool, CatBoostClassifier

cv_data = Pool(data=X_train, label=Y_train, cat_features=categorical_index, text_features=text_index)

params = {'loss_function': 'MultiClass', 
          'custom_metric': ['TotalF1:average=Weighted', 'TotalF1:average=Macro', 'TotalF1:average=Micro'],
          'iterations': 1000, #1000
          'learning_rate': 0.03, #0.03
          'random_seed': 40, #0
          'task_type': 'GPU'}

scores, models = cv(cv_data, params, fold_count=10, return_models=True)

Training on fold [0/10]
0:	learn: 1.8205396	test: 1.8205308	best: 1.8205308 (0)	total: 32.1ms	remaining: 32.1s
1:	learn: 1.7154847	test: 1.7153366	best: 1.7153366 (1)	total: 53.6ms	remaining: 26.7s
2:	learn: 1.6244204	test: 1.6243798	best: 1.6243798 (2)	total: 75.1ms	remaining: 25s
3:	learn: 1.5452401	test: 1.5450036	best: 1.5450036 (3)	total: 98.9ms	remaining: 24.6s
4:	learn: 1.4751726	test: 1.4747836	best: 1.4747836 (4)	total: 121ms	remaining: 24s
5:	learn: 1.4103070	test: 1.4099864	best: 1.4099864 (5)	total: 142ms	remaining: 23.6s
6:	learn: 1.3535417	test: 1.3531344	best: 1.3531344 (6)	total: 164ms	remaining: 23.2s
7:	learn: 1.3002352	test: 1.3001067	best: 1.3001067 (7)	total: 185ms	remaining: 23s
8:	learn: 1.2508298	test: 1.2508572	best: 1.2508572 (8)	total: 207ms	remaining: 22.8s
9:	learn: 1.2056029	test: 1.2057715	best: 1.2057715 (9)	total: 228ms	remaining: 22.6s
10:	learn: 1.1643919	test: 1.1644111	best: 1.1644111 (10)	total: 271ms	remaining: 24.4s
11:	learn: 1.1270642	test: 1.1

In [30]:
#Merge testing data and generate features

test = pd.merge(left=work_test, right=info_test, how='left', on='id_bh')

test_filter = test.sort_values(
    by=['id_bh', 'to_date', 'from_date'], 
    ascending=[True, False, False]
).drop_duplicates(
    subset=['id_bh'], 
    keep='first', 
    ignore_index=True
)

test_filter['age'] = 2022 - test_filter['bithYear'].astype(int)

test_filter['latest'] = test_filter['to_date'].astype('string').str.slice(0, 4)
test_filter['gap'] = 2022 - test_filter['latest'].astype(int)

test_filter.head()

,id,id_bh,id_management,id_office,company_type,job/role,from_date,to_date,employee_lv,work_address,bithYear,gender,address,age,latest,gap
0,3895,100000688,105,TE1931E,-1,lai may,20190100,20220400,9.0,ha noi,1973,MALE,viet nam,49,2022,0
1,3131,100005503,107,TG14845,-1,nhan vien quan ly,20200100,20220400,9.0,hn,1971,FEMALE,viet nam,51,2022,0
2,2828,100006307,105,HW05886,6,thieu,20120300,20220400,16.0,viet nam,1952,MALE,viet nam,70,2022,0
3,3865,100008625,106,QW02899,-1,thieu,20180100,20220400,15.0,viet nam,1973,MALE,ha noi,49,2022,0
4,3841,100009238,4015,TO08665,-1,lao dong pho thong,20200100,20211200,6.0,tinh nghe an,1978,MALE,nghe an,44,2021,1


In [31]:
#Configure testing data for CatBoost models

# test_data = test_filter[['id_management', 'id_office', 'company_type', 'employee_lv', 'gender', 'age', 'gap']] #no text features
test_data = test_filter[['id_management', 'id_office', 'company_type', 'employee_lv', 'gender', 'age', 'gap', 'work_address', 'address', 'job/role']] #include text features

In [32]:
#Predict testing data using trained models (type = Probability) and generate submission file

prediction_1 = models[0].predict(test_data, prediction_type='Probability')
prediction_2 = models[1].predict(test_data, prediction_type='Probability')
prediction_3 = models[2].predict(test_data, prediction_type='Probability')
prediction_4 = models[3].predict(test_data, prediction_type='Probability')
prediction_5 = models[4].predict(test_data, prediction_type='Probability')
prediction_6 = models[5].predict(test_data, prediction_type='Probability')
prediction_7 = models[6].predict(test_data, prediction_type='Probability')
prediction_8 = models[7].predict(test_data, prediction_type='Probability')
prediction_9 = models[8].predict(test_data, prediction_type='Probability')
prediction_10 = models[9].predict(test_data, prediction_type='Probability')

# avg_prediction = (prediction_1 + prediction_2 + prediction_3)/3
# avg_prediction = (prediction_1 + prediction_2 + prediction_3 + prediction_4 + prediction_5)/5
avg_prediction = (prediction_1 + prediction_2 + prediction_3 + prediction_4 + prediction_5 + prediction_6 + prediction_7 + prediction_8 + prediction_9 + prediction_10)/10


test_filter['label'] = (avg_prediction.argmax(axis=1)+1) #using argmax to generate label

result = test_filter[['id_bh', 'label']]
output = pd.merge(label_test, result, 'inner', 'id_bh')
output.to_csv('submit-v0.csv', index=False)

In [33]:
#Predict testing data using trained models (type = Class) and generate submission file

# prediction_class_1 = models[0].predict(test_data, prediction_type='Class')
# test_filter['label'] = (prediction_class_1)
# result = test_filter[['id_bh', 'label']]
# output = pd.merge(label_test, result, 'inner', 'id_bh')
# output.to_csv('prediction_class1-v3.csv', index=False)

# prediction_class_2 = models[1].predict(test_data, prediction_type='Class')
# test_filter['label'] = (prediction_class_2)
# result = test_filter[['id_bh', 'label']]
# output = pd.merge(label_test, result, 'inner', 'id_bh')
# output.to_csv('prediction_class2-v8.csv', index=False)

# prediction_class_3 = models[2].predict(test_data, prediction_type='Class')
# test_filter['label'] = (prediction_class_3)
# result = test_filter[['id_bh', 'label']]
# output = pd.merge(label_test, result, 'inner', 'id_bh')
# output.to_csv('prediction_class3-v3.csv', index=False)

# prediction_class_4 = models[3].predict(test_data, prediction_type='Class')
# test_filter['label'] = (prediction_class_4)
# result = test_filter[['id_bh', 'label']]
# output = pd.merge(label_test, result, 'inner', 'id_bh')
# output.to_csv('prediction_class4-v0.csv', index=False)

# prediction_class_5 = models[4].predict(test_data, prediction_type='Class')
# test_filter['label'] = (prediction_class_5)
# result = test_filter[['id_bh', 'label']]
# output = pd.merge(label_test, result, 'inner', 'id_bh')
# output.to_csv('prediction_class5-v0.csv', index=False)

In [34]:
#Vectorize training data for XGBoost or LightGBM

# vec = DictVectorizer()

# train_data = train_filter[['id_management', 'id_office', 'company_type', 'employee_lv', 'gender', 'age', 'gap', 'work_address', 'address', 'job/role']].to_dict('records')
# X_train = vec.fit_transform(train_data).toarray()

# Y_train = train_filter['label']

In [35]:
#Vectorize testing data

# test_data = test_filter[['id_management', 'id_office', 'company_type', 'employee_lv', 'gender', 'age', 'gap', 'work_address', 'address', 'job/role']].to_dict('records')
# X_test = vec.transform(test_data).toarray()

# test_id = info_test['id_bh'].tolist()

In [36]:
#Configure LightGBM and generate training, validation data

# import lightgbm as lgb

# params = {
#     'num_leaves': 80
#     }

# clf = lgb.LGBMClassifier(**params)

# X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.33, random_state=42)
# clf.fit(X_train, Y_train, eval_set=(X_valid, Y_valid))

In [37]:
#Configure XGBoost and generate training, validation data

# from xgboost import XGBClassifier

# clf = XGBClassifier()
# # Add silent=True to avoid printing out updates with each cycle
# X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.33, random_state=42)
# clf.fit(X_train, Y_train, verbose=False)

In [38]:
#Generate training and validation score

# print('Training set score: {:.4f}'.format(clf.score(X_train, Y_train)))
# print('Validation set score: {:.4f}'.format(clf.score(X_valid, Y_valid)))

In [39]:
#Predict test set

# prediction = clf.predict(X_test).astype('int64')

In [40]:
#Generate submission file

# output = pd.DataFrame({'id_bh': test_id, 'label': prediction})
# output.to_csv('prediction-v*.csv', index=False)